In [1]:
import pandas as pd #handle files, database etc.

In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv") #read the csv file using pandas library

dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
#data preprocessing - because categorial data can't be handle so we modify the data into numerical data using one hot encoding(Nominal)

dataset = pd.get_dummies(dataset, drop_first=True) #"drop_first=True" removes 1st category of each categorical column

dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [4]:
boolean_columns = dataset.select_dtypes(include=['bool']).columns #get boolean type column data from the dataset and store it in boolean_columns variable
#print(boolean_columns)

dataset[boolean_columns] = dataset[boolean_columns].astype(int) #convert boolean column dataset to 0's and 1's
#print(dataset[boolean_columns])

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset = dataset.drop("User ID", axis=1) #removing "User ID" feature using drop method. "axis=1" represent removing the "COLUMN"

dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [13]:
dataset["Purchased"].value_counts() #value_counts method calculates the purchased and not purchased value of "Purchased" column

Purchased
0    257
1    143
Name: count, dtype: int64

In [15]:
dataset.columns #return column names present in the dataset variable

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [17]:
independent = dataset[['Age', 'EstimatedSalary', 'Gender_Male']] #input features

dependent = dataset[['Purchased']] #output feature

In [19]:
#split training and testing dataset

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size=0.30, random_state=0)

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

parameters = {'C':[10,50,100],
             'penalty':['l1','l2','elasticnet'],
             'solver' : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
             'multi_class' : ['auto', 'ovr', 'multinomial']}

grid = GridSearchCV(LogisticRegression(), parameters, refit=True, n_jobs=-1, scoring='f1', verbose=3)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
495 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1194, in fit
    solver =

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [10, 50, 100],
                         'multi_class': ['auto', 'ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1', verbose=3)

In [21]:
classifier = grid.cv_results_

#print("CV_RESULTS : ", re)

grid_pred = grid.predict(X_test) #test the model with test data

from sklearn.metrics import confusion_matrix #Evaluation metrics

cm = confusion_matrix(y_test, grid_pred)
print("Confusion Matrix : ", cm) 

print("Best Score:", grid.best_score_)
print("Best Parameters:", grid.best_params_)

Confusion Matrix :  [[74  5]
 [ 8 33]]
Best Score: 0.7316852809876065
Best Parameters: {'C': 10, 'multi_class': 'auto', 'penalty': 'l2', 'solver': 'lbfgs'}


In [22]:
table = pd.DataFrame.from_dict(classifier).dropna() #convert dictonary format to DataFrame(table) and dropna() method is used to avoid to display nan values rows

table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_multi_class,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,0.019659,0.002708,0.016001,0.001802,10,auto,l1,liblinear,"{'C': 10, 'multi_class': 'auto', 'penalty': 'l...",0.756757,0.717949,0.400000,0.9,0.842105,0.723362,0.173786,3
5,0.021855,0.005053,0.015065,0.006323,10,auto,l1,saga,"{'C': 10, 'multi_class': 'auto', 'penalty': 'l...",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,37
6,0.098945,0.012603,0.019650,0.002660,10,auto,l2,lbfgs,"{'C': 10, 'multi_class': 'auto', 'penalty': 'l...",0.756757,0.717949,0.400000,0.9,0.883721,0.731685,0.180154,1
7,0.013840,0.007167,0.016824,0.001284,10,auto,l2,liblinear,"{'C': 10, 'multi_class': 'auto', 'penalty': 'l...",0.000000,0.000000,0.322581,0.0,0.090909,0.082698,0.125002,31
8,0.160115,0.014387,0.018007,0.002227,10,auto,l2,newton-cg,"{'C': 10, 'multi_class': 'auto', 'penalty': 'l...",0.756757,0.717949,0.400000,0.9,0.842105,0.723362,0.173786,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,0.018991,0.003334,0.015972,0.003475,100,multinomial,l1,saga,"{'C': 100, 'multi_class': 'multinomial', 'pena...",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,37
150,0.038024,0.004637,0.015275,0.004770,100,multinomial,l2,lbfgs,"{'C': 100, 'multi_class': 'multinomial', 'pena...",0.756757,0.717949,0.400000,0.9,0.842105,0.723362,0.173786,3
152,0.196092,0.020706,0.013433,0.003335,100,multinomial,l2,newton-cg,"{'C': 100, 'multi_class': 'multinomial', 'pena...",0.756757,0.717949,0.400000,0.9,0.842105,0.723362,0.173786,3
154,0.013508,0.004349,0.014629,0.003738,100,multinomial,l2,sag,"{'C': 100, 'multi_class': 'multinomial', 'pena...",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,37


In [23]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_pred) #generation classification report

print(clf_report)

              precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [24]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [25]:
#save the model using pickle library

import pickle
fileName = "finalized_model_logistic_classification_grid.sav"
pickle.dump(grid, open(fileName, "wb")) #saved model available in regressor variable. Now storing that variable into fileName

In [26]:
loaded_model = pickle.load(open("finalized_model_logistic_classification_grid.sav", "rb"))
result = loaded_model.predict([[30, 70000, 0]])

result

C:\Users\sethu\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([0], dtype=int64)